In [ ]:
#importing packages
import re

import numpy as np
import pandas as pd


from bioio import BioImage
import bioio_ome_tiff #using TIFF not czi files
from bioio_ome_tiff import Reader as OME_TIFFReader



from bioio import BioImage
import bioio_ome_tiff #using TIFF not czi files
from bioio_ome_tiff import Reader as OME_TIFFReader


from scipy.optimize import curve_fit
from scipy.optimize import brentq
from scipy.stats import kruskal

from matplotlib import pyplot as plt

from IPython.display import clear_output

import time
import gc


from scipy.optimize import curve_fit
from pathlib import Path


from natsort import natsorted

import seaborn as sns

import scipy.signal as signal

import os


import numpy as np

import itertools


from scikit_posthocs import posthoc_dunn
from scipy.stats import shapiro, levene
import scikit_posthocs as sp

import gc

from pathlib import Path
import shutil
import re
import tifffile as tiff

from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import closing, rectangle

from skimage.filters import threshold_otsu, median
from skimage.morphology import disk, remove_small_objects

import time
import re
import tifffile as tiff
from pathlib import Path
from bioio_ome_tiff import Reader as OME_TIFFReader
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt



In [ ]:
#PART 1: fix all file names to make sure they all match, drop all files without a image pair. This isn't necessary if file naming is consistent.

In [ ]:
#Fixing file name, function
def fix_exp(exp_data: Path, exp_tiff_data: Path):
    exp_tiff_data.mkdir(parents=True, exist_ok=True)           #make new file for .tif files
    for file in exp_data.rglob("*.tif"):
        match = re.search(r'(\d+pg)', file.name.lower())
        if match and match.group(1) == '0pg':                #skip the 0pg images
            print(f"skipping {file.name} only 0pg")
            continue

        new_name = file.name
        new_name = new_name.replace("26-", "25-")            #fixing misprint on date of 2026 to 2025
        new_name = new_name.replace("400pg", "480pg")        #fixing misprint on pg of 400 to 480

        corrected_path = exp_tiff_data / new_name

        try:                                                #list which files got their name corrected and add them to the new folder
            img = tiff.imread(file)
            tiff.imwrite(corrected_path, img)

            if new_name != file.name:
                print(f"Renamed and saved: {file.name} → {corrected_path.name}")
            else:
                print(f"No renaming needed for: {file.name}")

        except Exception as e:
            print(f"Failed to save {file.name} to {corrected_path.name}: {e}")
    print(f"done")
    return(exp_tiff_data)

In [ ]:
#Using the above function of the HJ mRNA
#HJ
#give path to files
HJ_data = Path('/Users/coral/Documents/data/HC_mRNA_October/October/HJ_control')
HJ_tiff_data = Path('/Users/coral/Documents/data/HC_mRNA_October/October/HJ_TIFF')

#run function
fix_exp(HJ_data, HJ_tiff_data)


In [ ]:
#Using the above function of the HC2 mRNA
#HC2
#give the path to the files
HC2_data = Path('/Users/coral/Documents/data/HC_mRNA_October/October/HC-2')
HC2_tiff_data = Path('/Users/coral/Documents/data/HC_mRNA_October/October/HC-2_TIFF')

#run function
fix_exp(HC2_data, HC2_tiff_data)

In [ ]:
#Using the above function of the HC3 mRNA
#HC3
#give path to files
HC3_data = Path('/Users/coral/Documents/data/HC_mRNA_October/October/HC-3')
HC3_tiff_data = Path('/Users/coral/Documents/data/HC_mRNA_October/October/HC-3_TIFF')
#run function
fix_exp(HC3_data, HC3_tiff_data)

In [ ]:
#Using the above function of the HC4 mRNA
#HC4
#give path to files
HC4_data = Path('/Users/coral/Documents/data/HC_mRNA_October/October/HC-4_Nov')
HC4_tiff_data = Path('/Users/coral/Documents/data/HC_mRNA_October/October/HC-4_TIFF')

#run function
fix_exp(HC4_data, HC4_tiff_data)

In [ ]:
#Using the above function of the HC6 mRNA
#HC6
#give path to files
HC6_data = Path('/Users/coral/Documents/data/HC_mRNA_October/October/HC-6')
HC6_tiff_data = Path('/Users/coral/Documents/data/HC_mRNA_October/October/HC-6_TIFF')

#run function
fix_exp(HC6_data, HC6_tiff_data)

In [ ]:
#PART 2: Uploading the blank background images for background subtraction.

In [ ]:
#Getting images from path
HC_data = Path('/Users/coral/Documents/data/HC_mRNA_October/HC_October_TIFFs')
bg_files = [f for f in HC_data.rglob("*.tif") if "BLANK" in f.name.upper()]    #my bg imgs have BLANK in the file, so find those and store them
green_imgs = []
red_imgs = []

for file in bg_files:                      #Assign the background images 488 or 561 channel
    reader = OME_TIFFReader(str(file))
    img = reader.get_image_data("CYX").astype(np.float32)
    if "488" in file.name:
        green_imgs.append(img)
    elif "561" in file.name:
        red_imgs.append(img)

bg_green = np.squeeze(np.mean(np.stack(green_imgs, axis=0), axis=0).astype(np.float32) if green_imgs else np.nan)  #mean all the intensities of the bg img for each background
bg_red = np.squeeze(np.mean(np.stack(red_imgs, axis=0), axis=0).astype(np.float32) if red_imgs else np.nan)


print(f"Background mean intensity - green:{bg_green}, red: {bg_red}")

In [ ]:
#Part3: Making directories for each experimental condition

In [ ]:
def process_experiment(exp_name: str, tiff_dir: Path, date_to_day: dict):
    """
    Process all TIFF images for an experiment to extract metadata such as
    average pixel intensity, channel (green/red), pg level, and experimental day.

    Parameters
    ----------
    exp_name : str
        Experiment name (e.g., "HJ")
    tiff_dir : Path
        Directory containing experiment TIFFs
    date_to_day : dict
        Mapping of dates to experimental days for this experiment

    Returns
    -------
    dict
        Nested dictionary of metadata: {pg: {"green": [...], "red": [...]}}
    """

    print(f"Processing experiment: {exp_name}")
    stack_dir = Path(tiff_dir)
    exp_stk_img_metadata = {}

    # Collect TIFFs
    file_list = [f for f in stack_dir.iterdir() if f.suffix.lower() == ".tif"]
    print(f"Found {len(file_list)} TIFFs in {stack_dir}")

    for idx, file in enumerate(file_list, start=1):
        print(f"\n[{idx}/{len(file_list)}] Processing: {file.name}", flush=True)

        # Skip blanks/controls
        if "BLANK" in file.name.upper():
            print(f"Skipping control/blank file: {file.name}")
            continue

        # Extract pg (e.g., 120pg)
        pg_parts = file.name.split("_")
        if len(pg_parts) < 3:
            print(f"Skipping file with unexpected name: {file.name}")
            continue
        pg = pg_parts[2]

        # Initialize structure for each pg
        if pg not in exp_stk_img_metadata:
            exp_stk_img_metadata[pg] = {"green": [], "red": []}

        # Determine channel
        if "488" in file.name:
            channel = "green"
        elif "561" in file.name:
            channel = "red"
        else:
            print(f"Skipping file (no valid channel): {file.name}")
            continue

        # Determine experimental day
        date_match = re.search(r"(\d{2}-\d{2}-\d{2})", file.name)
        date_str = date_match.group(1) if date_match else None
        day = date_to_day.get(exp_name, {}).get(date_str, "Unknown")

        # Load image safely
        start_time = time.time()
        try:
            img_data = tiff.imread(file)
            if img_data.ndim > 3:
                raise ValueError("multi-dimensional TIFF, switching reader")
            img_data = np.squeeze(img_data).astype(np.float32)
        except Exception as e:
            print(f"tifffile failed, switching to OME_TIFFReader: {e}")
            try:
                reader = OME_TIFFReader(str(file))
                img_data = reader.get_image_data("CYX", Z=0, T=0).astype(np.float32)
                img_data = np.squeeze(img_data)
            except Exception as e2:
                print(f"Failed to read {file.name}: {e2}")
                continue
            finally:
                reader = None

        elapsed = time.time() - start_time
        print(f"{file.name} loaded in {elapsed:.2f}s")

        # Compute average pixel intensity
        img_mean = float(np.mean(img_data))

        exp_stk_img_metadata[pg][channel].append({
            "file": str(file),
            "exp": exp_name,
            "day": day,
            "avg_intensity": img_mean
        })

        del img_data
        gc.collect()

    # Optional rename: normalize naming (e.g., 400pg → 480pg)
    cleaned_metadata = {}
    for pg, channels in exp_stk_img_metadata.items():
        new_pg = pg.replace("400pg", "480pg")
        cleaned_metadata[new_pg] = channels

    print(f"Done processing {len(file_list)} files for {exp_name}.")
    return cleaned_metadata


In [ ]:
#HJ directory
#path to tiffs
HJ_tiff_data = Path('/Users/coral/Documents/data/HC_mRNA_October/October/HJ_TIFF')

#date to day for HJ
date_to_day = {
    "HJ": {
        "25-10-06": "Day 1",
        "25-10-07": "Day 2",
        "25-10-08": "Day 3",
        "25-10-09": "Day 4",
        "25-10-10": "Day 5"
    }
}
#run function
HJ_stk_img_metadata = process_experiment("HJ", HJ_tiff_data, date_to_day)

In [ ]:
#HC2 directory
#path to tiffs
HC2_tiff_data = Path('/Users/coral/Documents/data/HC_mRNA_October/October/HC-2_TIFF')

#date to day for HC2
date_to_day = {
    "HC2": {
        "25-10-14": "Day 1",
        "25-10-15": "Day 2",
        "25-10-16": "Day 3",
        "25-10-17": "Day 4",
        "25-10-18": "Day 5"
    }
}
#run function
HC2_stk_img_metadata = process_experiment("HC2", HC2_tiff_data, date_to_day)

In [ ]:
#HC3 directory
#path to tiffs
HC3_tiff_data = Path('/Users/coral/Documents/data/HC_mRNA_October/October/HC-3_TIFF')

#date to day for HC3
date_to_day = {
    "HC3": {
        "25-10-21": "Day 1",
        "25-10-22": "Day 2",
        "25-10-23": "Day 3",
        "25-10-24": "Day 4",
        "25-10-25": "Day 5"
    }
}
#run function
HC3_stk_img_metadata = process_experiment("HC3", HC3_tiff_data, date_to_day)

In [ ]:
#HC4 directory
#path to tiffs
HC4_tiff_data = Path('/Users/coral/Documents/data/HC_mRNA_October/October/HC-4_TIFF')

#date to day for HC4
date_to_day = {
    "HC4": {
        "25-11-04": "Day 1",
        "25-11-05": "Day 2",
        "25-11-06": "Day 3",
        "25-11-07": "Day 4",
        "25-11-08": "Day 5"
    }
}
#run function
HC4_stk_img_metadata = process_experiment("HC4", HC4_tiff_data, date_to_day)

In [ ]:
#HC6 directory
#path to tiffs
HC6_tiff_data = Path('/Users/coral/Documents/data/HC_mRNA_October/October/HC-6_TIFF')

#date to day for HC6
date_to_day = {
    "HC6": {
        "25-10-14": "Day 1",
        "25-10-15": "Day 2",
        "25-10-16": "Day 3",
        "25-10-17": "Day 4",
        "25-10-18": "Day 5"
    }
}
#run function
HC6_stk_img_metadata = process_experiment("HC6", HC6_tiff_data, date_to_day)

In [ ]:
#Part4: Correcting images with Background subtraction and normalizing to 561

In [ ]:
def correct_images(stk_img_metadata, bg_green, bg_red, output_dir, saturation_value=65535):
    """
    Apply background correction, saturation masking, and compute average intensity
    for all images in an experiment.

    Parameters
    ----------
    stk_img_metadata : dict
        Metadata structure {pg: {"green": [...], "red": [...]}}
    bg_green : np.ndarray
        Background image for green channel
    bg_red : np.ndarray
        Background image for red channel
    output_dir : Path or str
        Directory to save corrected images
    saturation_value : int, optional
        Saturation threshold (default 65535 for 16-bit images)

    Returns
    -------
    dict
        Updated metadata with corrected image file paths and avg_intensity
    """

    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    for pg, channels in stk_img_metadata.items():
        for channel, items in channels.items():
            for i, item in enumerate(items):
                file_path = Path(item["file"])

                # Load image
                img = tiff.imread(file_path).astype(np.float32)

                # Mask saturated pixels
                img[img == saturation_value] = np.nan

                # Background subtraction
                if channel == "green":
                    img = img - bg_green
                elif channel == "red":
                    img = img - bg_red

                # Mask nonpositive values
                img[img <= 0] = np.nan

                # Save corrected image
                corrected_file = output_dir / f"{pg}_{channel}_{i}_corrected.tif"
                tiff.imwrite(corrected_file, img.astype(np.float32))

                # Update metadata
                stk_img_metadata[pg][channel][i]["file_corr"] = str(corrected_file)
                stk_img_metadata[pg][channel][i]["avg_intensity"] = float(np.nanmean(img))

                del img
                gc.collect()

    print("✅ Correction and avg_intensity computation complete.")
    return stk_img_metadata


def normalize_red_to_green(stk_img_metadata):
    """
    Normalize red channel intensities relative to green channel for each pg.

    Parameters
    ----------
    stk_img_metadata : dict
        Metadata structure {pg: {"green": [...], "red": [...]}} with avg_intensity fields

    Returns
    -------
    dict
        Updated metadata with normalization factors and normalized file paths
    """
    for pg, channels in stk_img_metadata.items():
        green_items = channels.get("green", [])
        red_items = channels.get("red", [])

        print(f"\nNormalizing pg = {pg} | green: {len(green_items)} | red: {len(red_items)}")

        for i, red_item in enumerate(red_items):
            if i >= len(green_items):
                print(f"Skipping red[{i}] — no matching green image.")
                continue

            green_avg = green_items[i].get("avg_intensity", np.nan)
            red_avg = red_item.get("avg_intensity", np.nan)

            if np.isnan(green_avg) or green_avg == 0:
                print(f"Skipping pg={pg}, index={i}: invalid green_avg ({green_avg})")
                continue

            norm_factor = red_avg / green_avg
            red_item["norm_factor"] = norm_factor

            if np.isnan(norm_factor) or norm_factor == 0:
                print(f"Skipping pg={pg}, index={i}: invalid norm_factor ({norm_factor})")
                continue

            red_file_corr = red_item.get("file_corr", None)
            if red_file_corr:
                img_red = tiff.imread(red_file_corr).astype(np.float32)
                img_red_norm = img_red / norm_factor

                norm_file = Path(red_file_corr).with_name(
                    f"{Path(red_file_corr).stem}_norm.tif"
                )
                tiff.imwrite(norm_file, img_red_norm.astype(np.float32))
                red_item["file_norm"] = str(norm_file)

                del img_red, img_red_norm
                gc.collect()

        print(f"Done with {pg}")

    print("Red/green normalization complete.")
    return stk_img_metadata


In [ ]:
HJ_corrected_dir = Path("/Users/coral/Documents/data/HC_mRNA_October/October/HJ_corr")
HJ_stk_img_metadata = correct_images(
    stk_img_metadata=HJ_stk_img_metadata,
    bg_green=bg_green,
    bg_red=bg_red,
    output_dir=HJ_corrected_dir
)

HJ_stk_img_metadata = normalize_red_to_green(HJ_stk_img_metadata)


In [ ]:
HC2_corrected_dir = Path("/Users/coral/Documents/data/HC_mRNA_October/October/HC2_corr")
HC2_stk_img_metadata = correct_images(
    stk_img_metadata=HC2_stk_img_metadata,
    bg_green=bg_green,
    bg_red=bg_red,
    output_dir=HC2_corrected_dir
)

HC2_stk_img_metadata = normalize_red_to_green(HC2_stk_img_metadata)


In [ ]:
HC3_corrected_dir = Path("/Users/coral/Documents/data/HC_mRNA_October/October/HC3_corr")
HC3_stk_img_metadata = correct_images(
    stk_img_metadata=HC3_stk_img_metadata,
    bg_green=bg_green,
    bg_red=bg_red,
    output_dir=HC3_corrected_dir
)

HC3_stk_img_metadata = normalize_red_to_green(HC3_stk_img_metadata)


In [ ]:
HC4_corrected_dir = Path("/Users/coral/Documents/data/HC_mRNA_October/October/HC4_corr")
HC4_stk_img_metadata = correct_images(
    stk_img_metadata=HC4_stk_img_metadata,
    bg_green=bg_green,
    bg_red=bg_red,
    output_dir=HC4_corrected_dir
)

HC4_stk_img_metadata = normalize_red_to_green(HC4_stk_img_metadata)


In [ ]:
HC6_corrected_dir = Path("/Users/coral/Documents/data/HC_mRNA_October/October/HC6_corr")
HC6_stk_img_metadata = correct_images(
    stk_img_metadata=HC6_stk_img_metadata,
    bg_green=bg_green,
    bg_red=bg_red,
    output_dir=HC6_corrected_dir
)

HC6_stk_img_metadata = normalize_red_to_green(HC6_stk_img_metadata)


In [ ]:
#Part5: Build Dataframe for each condition

In [ ]:
def build_avg_norm_df(stk_img_metadata, exp_name):
    rows = []
    for pg, channels in stk_img_metadata.items():
        red_items = channels.get("red", [])
        for item in red_items:
            avg_norm = item.get("avg_intensity", np.nan)  # use already stored value
            day = item.get("day", "Unknown")
            rows.append({
                "exp": exp_name,
                "pg": pg,
                "day": day,
                "avg_norm_intensity": avg_norm
            })
    df = pd.DataFrame(rows)
    print(f"✅ DataFrame built for {exp_name} ({len(df)} entries).")
    return df


In [ ]:
#HJ dataframe
df_HJ = build_avg_norm_df(HJ_stk_img_metadata, "HJ")
df_HJ.head()

In [ ]:
#HC2 dataframe
df_HC2 = build_avg_norm_df(HC2_stk_img_metadata, "HC2")
df_HC2.head()

In [ ]:
#HC3 dataframe
df_HC3 = build_avg_norm_df(HC3_stk_img_metadata, "HC3")
df_HC3.head()

In [ ]:
#HC4 dataframe
df_HC4 = build_avg_norm_df(HC4_stk_img_metadata, "HC4")
df_HC4.head()

In [ ]:
#HC6 dataframe
df_HC6 = build_avg_norm_df(HC6_stk_img_metadata, "HC6")
df_HC6.head()

In [ ]:
#Part6: Combine all the dataframe for plotting and fold change

In [ ]:
# Combine all experiment DataFrames
all_exps = {
    "HJ": df_HJ,
    "HC2": df_HC2,
    "HC3": df_HC3,
    "HC4": df_HC4,
    "HC6": df_HC6,
}

# Combine all into one big DataFrame with an experiment column
combined_df = pd.concat(
    [df.assign(experiment=name) for name, df in all_exps.items()],
    ignore_index=True
)

# Keep consistent experiment color palette
exp_palette = dict(zip(all_exps.keys(), sns.color_palette("tab10", n_colors=len(all_exps))))


In [ ]:
#Part7: Getting the average baseline of the control ( 120pg HJ) for Fold change analysis

In [ ]:
#120 baseline
import pandas as pd

# --- Baseline: get HJ averages per pg & day ---
baseline_HJ_120 = (
    combined_df[
        (combined_df["experiment"] == "HJ") & (combined_df["pg"] == "120pg")
    ]
    .groupby(["day"], as_index=False)["avg_norm_intensity"]
    .mean()
    .rename(columns={"avg_norm_intensity": "HJ_120pg_intensity"})
)

# --- Merge HJ baseline back onto the combined data ---
merged = combined_df.merge(baseline_HJ_120, on=["day"], how="left")

# --- Compute fold change relative to HJ ---
merged["fold_change_v_120pg"] = merged["avg_norm_intensity"] / merged["HJ_120pg_intensity"]

# --- Filter for only HC2 and HC6 (the comparisons you care about) ---
fc_df_v_HJ_120pg = merged[merged["experiment"].isin(["HC2", "HC3","HC4","HC6"])]

# Optional: sanity check
print(fc_df_v_HJ_120pg[["experiment", "pg", "day", "avg_norm_intensity", "HJ_120pg_intensity", "fold_change_v_120pg"]].head())


In [ ]:
#Part8: Plotting the fold change of each condition to 120 pg of HJ

In [ ]:
#fold change to HJ 120
fc_df_v_HJ_120pg = fc_df_v_HJ_120pg.copy()
experiment_order = ["HC6","HC2","HC3","HC4"]
palette = {
    "HC2": "#2ecc71",
    "HC6": "#3498db",
    "HC4": "#ff0000",
    "HC3": "#ffa500",
}
# Normalize day strings and extract numeric day (works if day values are "1", "Day 1", "day_1", "day1", etc.)
fc_df_v_HJ_120pg["day_str"] = fc_df_v_HJ_120pg["day"].astype(str).str.strip().str.title().str.replace("_", " ")
fc_df_v_HJ_120pg["day_num"] = fc_df_v_HJ_120pg["day_str"].str.extract(r"(\d+)").astype(int)

# Sort by numeric day so lines draw in correct order
fc_df_v_HJ_120pg = fc_df_v_HJ_120pg.sort_values("day_num")

# Plot with numeric x, then set human-friendly xticks
plt.figure(figsize=(8,5))
sns.lineplot(
    data=fc_df_v_HJ_120pg,
    x="day_num",           # numeric ordering
    y="fold_change_v_120pg",
    hue="experiment",
    hue_order=experiment_order,
    marker="o",
    linewidth=2,
    palette=palette
)

plt.axhline(1, color="gray", linestyle="--", label="HJ baseline")
plt.xticks([1,2,3,4,5], ["Day 1","Day 2","Day 3","Day 4","Day 5"])
plt.xlim(0.8, 5.2)  # small padding
plt.ylim(0,12)
plt.ylabel("Fold Change (vs HJ)")
plt.xlabel("Day")
plt.title("Fold Change of HC Experiments Relative to HJ 120pg Over Time")
plt.legend(title="Experiment")
plt.tight_layout()
plt.savefig("/Users/coral/Documents/data/HC_mRNA_October/11-09-25_graphs_pg/fold_change_plot_HJ120.pdf")
plt.show()


In [ ]:
#Plotting the fold change of the best conditions, HC2 and HC6.
fc_df_HC2_HC6_v_HJ_120 = fc_df_v_HJ_120pg[fc_df_v_HJ_120pg["experiment"].isin(["HC2", "HC6"])].copy()


# Normalize day strings and extract numeric day (works if day values are "1", "Day 1", "day_1", "day1", etc.)
fc_df_HC2_HC6_v_HJ_120["day_str"] = fc_df_HC2_HC6_v_HJ_120["day"].astype(str).str.strip().str.title().str.replace("_", " ")
fc_df_HC2_HC6_v_HJ_120["day_num"] = fc_df_HC2_HC6_v_HJ_120["day_str"].str.extract(r"(\d+)").astype(int)

# Sort by numeric day so lines draw in correct order
fc_df_HC2_HC6_v_HJ_120 = fc_df_HC2_HC6_v_HJ_120.sort_values("day_num")
palette = {
    "HC2": "#2ecc71",
    "HC6": "#3498db"
}
# Plot with numeric x, then set human-friendly xticks
plt.figure(figsize=(8,5))
sns.lineplot(
    data=fc_df_HC2_HC6_v_HJ_120,
    x="day_num",           # numeric ordering
    y="fold_change_v_120pg",
    hue="experiment",
    marker="o",
    linewidth=2,
    palette=palette
)

plt.axhline(1, color="gray", linestyle="--", label="HJ baseline")
plt.xticks([1,2,3,4,5], ["Day 1","Day 2","Day 3","Day 4","Day 5"])
plt.xlim(0.8, 5.2)  # small padding
plt.ylim(0,12)
plt.ylabel("Fold Change (vs HJ)")
plt.xlabel("Day")
plt.title("Fold Change of HC2 and HC6 Relative to HJ 120pg Over Time")
plt.legend(title="Experiment")
plt.tight_layout()
plt.savefig("/Users/coral/Documents/data/HC_mRNA_October/11-09-25_graphs_pg/HC2_HC6_fold_change_HJ120.pdf")
plt.show()


In [ ]:
#Fold change of HC2 and HC6 only at 480 pg.
fc_df_HC2_HC6_v_HJ_120 = fc_df_v_HJ_120pg[fc_df_v_HJ_120pg["experiment"].isin(["HC2", "HC6"])].copy()
#filtering
fc_df_HC2_HC6_v_HJ_120 = fc_df_HC2_HC6_v_HJ_120[
    fc_df_HC2_HC6_v_HJ_120["pg"] == "480pg"
].copy()


# Normalize day strings and extract numeric day (works if day values are "1", "Day 1", "day_1", "day1", etc.)
fc_df_HC2_HC6_v_HJ_120["day_str"] = fc_df_HC2_HC6_v_HJ_120["day"].astype(str).str.strip().str.title().str.replace("_", " ")
fc_df_HC2_HC6_v_HJ_120["day_num"] = fc_df_HC2_HC6_v_HJ_120["day_str"].str.extract(r"(\d+)").astype(int)

# Sort by numeric day so lines draw in correct order
fc_df_HC2_HC6_v_HJ_120 = fc_df_HC2_HC6_v_HJ_120.sort_values("day_num")
palette = {
    "HC2": "#2ecc71",
    "HC6": "#3498db"
}
# Plot with numeric x, then set human-friendly xticks
plt.figure(figsize=(8,5))
sns.lineplot(
    data=fc_df_HC2_HC6_v_HJ_120,
    x="day_num",           # numeric ordering
    y="fold_change_v_120pg",
    hue="experiment",
    marker="o",
    linewidth=2,
    palette=palette
)

plt.axhline(1, color="gray", linestyle="--", label="HJ baseline")
plt.xticks([1,2,3,4,5], ["Day 1","Day 2","Day 3","Day 4","Day 5"])
plt.xlim(0.8, 5.2)  # small padding
plt.ylim(0,12)
plt.ylabel("Fold Change (vs HJ)")
plt.xlabel("Day")
plt.title("Fold Change of HC2 and HC6 480 pg Relative to HJ 120pg Over Time")
plt.legend(title="Experiment")
plt.tight_layout()
plt.savefig("/Users/coral/Documents/data/HC_mRNA_October/11-09-25_graphs_pg/HC2_HC6_480_fold_change_HJ120.pdf")
plt.show()


In [ ]:
#Part9: Plotting the Death and abnormalty of the injection conditions

In [ ]:
# --- 1. Load the CSV ---
file_path = '/Users/coral/Documents/data/HC_mRNA_October/HC_mRNA_Death - wout_imaged.csv'  # update path if needed
df = pd.read_csv(file_path)

# --- 2. Preview structure (optional) ---
print(df.columns)


# --- 3. Combine totals ---
df["Total_Dead"] = df["day_1_death"] + df["day_2_death"]
df["Total_Abnormal"] = df["day_1_ab"] + df["day_2_ab"]

# --- 4. Compute percentages ---
df["% Dead"] = (df["Total_Dead"] / df["total_wout_imaged"]) * 100
df["% Abnormal"] = (df["Total_Abnormal"] / df["total_wout_imaged"]) * 100


# --- 5. Calculate "Normal" as what's left to reach 100% ---
df["% Normal"] = 100 - (df["% Dead"] + df["% Abnormal"])

df["exp_pg"] = df["exp"].astype(str) + "_" + df["pg"].astype(str)

# --- 6. Prepare for plotting ---
plot_df = df[["exp_pg", "% Dead", "% Abnormal", "% Normal"]]
plot_df = plot_df.set_index("exp_pg")

save_dir = Path("/Users/coral/Documents/data/HC_mRNA_October/bar_graphs")
save_dir.mkdir(parents=True, exist_ok=True)

# --- 7. Plot stacked bar chart ---
colors = ["#e74c3c", "#f39c12", "#2ecc71"]  # red, orange, blue, green

plot_df.plot(
    kind="bar",
    stacked=True,
    color=colors,
    edgecolor="black",
    figsize=(10,6)
)

plt.ylabel("% of Embryos")
plt.xlabel("exp + pg")
plt.title("Embryo Outcomes per Condition without Imaged Embryos")
plt.ylim(0, 100)
plt.legend(title="Outcome", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
save_path_png = save_dir / "Embryo_Outcomes_without_Imaged.png"
save_path_pdf = save_dir / "Embryo_Outcomes_without_Imaged.pdf"

plt.savefig(save_path_png, dpi=300)
plt.savefig(save_path_pdf, dpi=300)
plt.show()


